In [ ]:
import sys
sys.path.append("../")
sys.path.append("../../")

sys.dont_write_bytecode = True

import vectorized_loop as vec

import argparse
import time

import numpy as np
import torch
import torch.nn as nn

import pyro
import pyro.infer
import pyro.distributions as dist

The Parametric Model where its order differed by its parameters.
-----------------

One of main functionality of our method is to find an order of a model automatically, but someone may criticise that the order can be determined intuitively according to the structure of the model.

In this notebook, we will introduce the model of which we can modify the order of the model without changing structures.

The model is mathemetically same with an auto-regressive model, but keeps track of previous values weighted in a peculiar way.

given coefficients : $c_0, c_1, ... , c_m$

$x_0 = 1$

$h_0 = \text{zero\_vector}(m+1)$

$x_{n+1} \sim Dist(\text{sum}(h_n))$

$h_{n+1} = c \odot (x :: h_n[:-1])$

where $\odot$ is an element-wise product, and $::$ is a concatenation.

We can think coefficients $c_i$ as forgetting factors. As time goes by, the $i$ th previous value becomes smaller by a factor of $c_i$.

In [ ]:
@vec.vectorize
def model(s: vec.State, length:int, coeffs:torch.Tensor, vectorized = False):
    s.x = 1
    s.h = torch.zeros(len(coeffs))
    for i in vec.range("loop", length, vectorized = vectorized):
        s.x = pyro.sample("x", dist.Beta(1+s.h.sum(-1), 1+s.h.sum(-1)))
        s.h = coeffs*torch.cat((s.x[..., None], s.h[...,:-1]), dim = -1)

Thinking roughly, we can determine $m+1$ as an order of this model, since the model keeps track of $m+1$ previous values.

However, the order of the model can be much smaller than $m+1$ depending on the coefficients.

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

length = 30
coeffs1 = torch.tensor([1,1,1,1,1,1,1,1,1,1])
coeffs2 = torch.tensor([1,1,1,0,1,1,1,1,1,1])

logging.debug("coeffs1")
tr = vec.trace(model).get_trace(length, coeffs1)
tr2 = vec.trace(vec.replay(model, trace = tr)).get_trace(length, coeffs1, True)

logging.debug("coeffs2")
tr = vec.trace(model).get_trace(length, coeffs1)
tr2 = vec.trace(vec.replay(model, trace = tr)).get_trace(length, coeffs2, True)



DEBUG:root:coeffs1
DEBUG:vectorized_loop.markov_messenger:loop (vectorized): repeat 11
DEBUG:root:coeffs2
DEBUG:vectorized_loop.markov_messenger:loop (vectorized): repeat 4


The model with second coefficients has an order 3, because the forgetting factor 0 at 4th position makes every $4\leq$ th previous values zero.

I didn't make a proof, but the order of this model will be the smallest $n$ where $\prod_{i=0}^n c_i = 0$, which is not simple to be hard-coded. 